<a href="https://colab.research.google.com/github/EDMMaster/Project3G1/blob/elijah/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Terry

In [1]:
# Create directories for each dataset
!mkdir -p ./saurabh_fake_news_classification
!mkdir -p ./aadyasingh_fake_news_classification

# Download and unzip the first dataset into the specified folder
!kaggle datasets download -d saurabhshahane/fake-news-classification -p ./saurabh_fake_news_classification --unzip

# Download and unzip the second dataset into its specified folder
!kaggle datasets download -d aadyasingh55/fake-news-classification -p ./aadyasingh_fake_news_classification --unzip

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
License(s): Attribution 4.0 International (CC BY 4.0)
100% 92.0M/92.1M [00:01<00:00, 83.2MB/s]
100% 92.1M/92.1M [00:01<00:00, 73.2MB/s]
Dataset URL: https://www.kaggle.com/datasets/aadyasingh55/fake-news-classification
License(s): apache-2.0
 80% 31.0M/39.0M [00:00<00:00, 112MB/s] 
100% 39.0M/39.0M [00:00<00:00, 92.9MB/s]


In [2]:
import pandas as pd

In [3]:
# Load the datasets
wel_fake_df = pd.read_csv('./saurabh_fake_news_classification/WELFake_Dataset.csv')
train_df = pd.read_csv('./aadyasingh_fake_news_classification/train (2).csv', sep=';')
test_df = pd.read_csv('./aadyasingh_fake_news_classification/test (1).csv', sep=';')
eval_df = pd.read_csv('./aadyasingh_fake_news_classification/evaluation.csv', sep=';')

In [4]:
wel_fake_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [5]:
train_df.head()

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0


In [6]:
# Flip labels in WELFake_Dataset.csv
wel_fake_df['label'] = wel_fake_df['label'].apply(lambda x: 1 if x == 0 else 0)
wel_fake_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,0
1,1,NaN,Did they post their votes for Hillary already?,0
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",0
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,1
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",0


In [7]:
# Combine all datasets into a single DataFrame
combined_df = pd.concat([wel_fake_df, train_df, test_df, eval_df], ignore_index=True)
combined_df.shape

(112721, 4)

In [8]:
# Count duplicates based on "title" and "text" columns in the combined DataFrame
total_duplicates = combined_df.duplicated(subset=["title", "text"]).sum()
print("Total Duplicates Across All Datasets:", total_duplicates)

Total Duplicates Across All Datasets: 27054


In [9]:
deduplicated_df = combined_df.drop_duplicates(subset=["title", "text"], inplace=False)
deduplicated_df.shape

(85667, 4)

LaQuita

In [ ]:
!pip install vaderSentiment

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, accuracy_score

ModuleNotFoundError: No module named 'vaderSentiment'

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
print(deduplicated_df.isnull().sum())

In [ ]:
nulls_removed_df = deduplicated_df.dropna().reset_index(drop=True)
cleaned_df = nulls_removed_df.copy()
print(cleaned_df.isnull().sum())

In [ ]:
cleaned_df.head()

In [ ]:
X = cleaned_df.drop(columns=['label'])
y = cleaned_df['label']    

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# Mapping 0 to 'Fake' and 1 to 'Real'
label_mapping = {0: 'Fake', 1: 'Real'}

# Plotting the pie chart with custom labels
cleaned_df['label'].value_counts().rename(index=label_mapping).plot.pie(
    autopct='%.2f%%',
    labels=['Real', 'Fake']  # Explicitly set labels
)

plt.title("Label Distribution")
plt.show()

In [ ]:
# Count plot for label distribution
sns.countplot(data=cleaned_df, x='label', hue='label', legend=False, palette=['orange', 'blue'])
plt.title('Label Distribution')
plt.xlabel('Label')
plt.xticks([0, 1], ['Fake', 'Real'])
plt.ylabel('Count')
plt.show()

In [ ]:
# Donut plot for label distribution
sizes = cleaned_df['label'].value_counts()
colors = ['blue', 'orange']
explode = (0.1, 0)  # explode the first slice

plt.pie(sizes, labels=['Real', 'Fake'], autopct='%.2f%%', startangle=90, colors=colors, explode=explode)
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Label Distribution')
plt.show()

In [ ]:
# Vocab_size = Unique words in our Corpus (entire document)
vocab_size = 10000

In [ ]:
messages = X.copy()

# Reset the index to avoid errors caused by using dropna() earlier
messages.reset_index(inplace=True)
print(messages['title'][1])

In [ ]:

### Text Preprocessing ###
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Create an instance of the CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=5, stop_words='english')
dtm = cv.fit_transform(corpus)
print(dtm.shape)


# Inspect the corpus
print(corpus[:30])

# Calculate the maximum sentence length
max_length = max(len(sentence.split()) for sentence in corpus)
print("Maximum sentence length:", max_length)

# Convert Text to One Hot Representation and Add Padding
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
print(onehot_repr[:5])

sentence_length = 50
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_length)
print(embedded_docs)

In [ ]:
### LDA Integration ###
count_vectorizer = CountVectorizer(max_features=1000, stop_words='english')
dtm = count_vectorizer.fit_transform(corpus)

lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(dtm)

# Print the top 15 words for each topic
for index, topic in enumerate(lda.components_):
    print(f'The Top 15 Words For Topic #{index + 1}')
    print([count_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

# Transform our DTM
topic_results = lda.transform(dtm)

# Adding Topic Labels
topic_labels = {
    1: 'Entertainment',
    2: 'Sports',
    3: 'Business',
    4: 'Politics',
    5: 'Technology'
}

def add_topic_labels(cleaned_df, topic_results, topic_labels):
    cleaned_df['topic'] = topic_results.argmax(axis=1) + 1
    cleaned_df['topic_label'] = cleaned_df['topic'].map(topic_labels)

add_topic_labels(cleaned_df, topic_results, topic_labels)

# Display the first 20 rows of the updated DataFrame
print(cleaned_df.head(20))

In [ ]:
### VADER Sentiment Analysis ###
analyzer = SentimentIntensityAnalyzer()
sentiments = [analyzer.polarity_scores(title)['compound'] for title in corpus]

X_sentiment = np.array(sentiments).reshape(-1, 1)
X_final_with_topics = np.concatenate((embedded_docs, topic_results), axis=1)
X_final_with_sentiment = np.concatenate((X_final_with_topics, X_sentiment), axis=1)

# Convert to numpy arrays
X_final = np.array(X_final_with_sentiment)
y_final = np.array(y)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)
     

In [ ]:
# Define and train LSTM models
embedding_vector_features = 100

# First Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

# Evaluate First Model
y_log = model.predict(X_test)
y_pred = np.where(y_log > 0.5, 1, 0)
acc = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
print(acc)
print(confusion_mat)

In [ ]:
# Second Model with Multiple LSTM Layers
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_vector_features))
model1.add(LSTM(100, return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(50, return_sequences=True))
model1.add(Dropout(0.1))
model1.add(LSTM(20))
model1.add(Dropout(0.1))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

# Evaluate Second Model
y_log_1 = model1.predict(X_test)
y_pred_1 = np.where(y_log_1 > 0.5, 1, 0)
acc_1 = accuracy_score(y_test, y_pred_1)
confusion_mat_1 = confusion_matrix(y_test, y_pred_1)
print(acc_1)
print(confusion_mat_1)

In [ ]:
# Third Model with Bidirectional LSTM
embedding_vector_features = 150

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_vector_features))
model2.add(Bidirectional(LSTM(200)))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=120)

y_log_2 = model2.predict(X_test)
y_pred_2 = np.where(y_log_2 > 0.5, 1, 0)
acc_2 = accuracy_score(y_test, y_pred_2)
confusion_mat_2 = confusion_matrix(y_test, y_pred_2)
print(acc_2)
print(confusion_mat_2)
     

Elijah

In [ ]:
# Gradio
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [ ]:
# Load the trained LSTM model
model = load_model("model.h5")  # Replace with your model path

# Load the tokenizer
with open("tokenizer.pkl", "rb") as handle:
    tokenizer = pickle.load(handle)

# Predefined sequence length
MAX_SEQUENCE_LENGTH = 50  # Update as per the model training

In [ ]:
def classify_text(news_text):
    """
    Preprocess the text input, predict using the LSTM model, and return the classification.
    """
    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([news_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)

    # Predict
    prediction = model.predict(padded_sequence)[0][0]

    # Interpret the result
    result = "Fake News" if prediction >= 0.5 else "Real News"
    confidence = prediction if prediction >= 0.5 else 1 - prediction
    return f"{result} (Confidence: {confidence:.2%})"

In [ ]:
# Gradio interface
iface = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here...", label="Enter News Text"),
    outputs=gr.Textbox(label="Prediction"),
    title="Fake News Detector",
    description="This tool uses an LSTM model to classify whether a piece of text is fake or real news."
)

# Launch the app
iface.launch(share=True)